In [10]:
from amplpy import AMPL
import pandas as pd

ampl = AMPL()

ampl.read("../../ampl/models/model.mod")
ampl.read_data("../../data/data.dat")

solvers = ["highs", "gurobi"]
all_results = []

for solver in solvers:
    print(f"\n=== SOLVER: {solver.upper()} ===")
    ampl.option["solver"] = solver
    
    solver_results = []
    
    for p_val in range(1, 6):
        ampl.get_parameter("p").set(p_val)
        ampl.solve()

        solve_result = ampl.get_value("solve_result")
        if solve_result != "solved":
            print(f"p={p_val} → No se pudo resolver: {solve_result}")
            continue

        obj = ampl.get_objective("TotalCost").value()
        x_vals = ampl.get_variable("x").get_values().to_pandas()
        
        nodos_seleccionados = []
        for index, row in x_vals.iterrows():
            if row.iloc[0] >= 0.5:
                nodos_seleccionados.append(index)
        
        resultado = {
            'solver': solver,
            'p': p_val,
            'costo_total': obj,
            'nodos_cd': nodos_seleccionados,
            'num_nodos': len(nodos_seleccionados)
        }
        solver_results.append(resultado)
        all_results.append(resultado)
        
        print(f"p={p_val} → Costo: {obj:.2f}, CDs en nodos: {nodos_seleccionados}")
    
    print(f"\nResultados con {solver.upper()}:")
    df_solver = pd.DataFrame(solver_results)
    print(df_solver[['p', 'costo_total', 'num_nodos']].to_string(index=False))

print("\n=== RESUMEN COMPLETO ===")
df_all_results = pd.DataFrame(all_results)
print(df_all_results.to_string(index=False))


=== SOLVER: HIGHS ===
HiGHS 1.11.0HiGHS 1.11.0: optimal solution; objective 5483.5
0 simplex iterations
0 branching nodes
p=1 → Costo: 5483.50, CDs en nodos: [1]
HiGHS 1.11.0HiGHS 1.11.0: optimal solution; objective 2993.5
13 simplex iterations
1 branching nodes
p=2 → Costo: 2993.50, CDs en nodos: [1, 2]
HiGHS 1.11.0HiGHS 1.11.0: optimal solution; objective 2170.5
12 simplex iterations
1 branching nodes
p=3 → Costo: 2170.50, CDs en nodos: [1, 2, 4]
HiGHS 1.11.0HiGHS 1.11.0: optimal solution; objective 1462.5
11 simplex iterations
1 branching nodes
p=4 → Costo: 1462.50, CDs en nodos: [1, 2, 4, 5]
HiGHS 1.11.0HiGHS 1.11.0: optimal solution; objective 1020
10 simplex iterations
1 branching nodes
p=5 → Costo: 1020.00, CDs en nodos: [1, 2, 3, 4, 5]

Resultados con HIGHS:
 p  costo_total  num_nodos
 1       5483.5          1
 2       2993.5          2
 3       2170.5          3
 4       1462.5          4
 5       1020.0          5

=== SOLVER: GUROBI ===
Gurobi 12.0.2:Gurobi 12.0.2: optimal

In [11]:
df_all_results

,solver,p,costo_total,nodos_cd,num_nodos
0,highs,1,5483.5,[1],1
1,highs,2,2993.5,"[1, 2]",2
2,highs,3,2170.5,"[1, 2, 4]",3
3,highs,4,1462.5,"[1, 2, 4, 5]",4
4,highs,5,1020.0,"[1, 2, 3, 4, 5]",5
5,gurobi,1,5483.5,[1],1
6,gurobi,2,2993.5,"[1, 2]",2
7,gurobi,3,2170.5,"[1, 2, 4]",3
8,gurobi,4,1462.5,"[1, 2, 4, 5]",4
9,gurobi,5,1020.0,"[1, 2, 3, 4, 5]",5


In [12]:
df_all_results.to_csv("../../results/solver_results.csv", index=False)